# PyTorch BigBird NER Baseline - CV 0.615
This notebook is a PyTorch starter notebook for Kaggle's "Feedback Prize - Evaluating Student Writing" Competition. It demonstrates how to train, infer, and submit a model to Kaggle without internet. Currently this notebook uses

* backbone BigBird  (with HuggingFace's head for TokenClassification)
* NER formulation (with `is_split_into_words=True` tokenization)
* one fold

By changing a few lines of code, we can use this notebook to evaluate different PyTorch backbones! And we can run all sorts of other experiments. If we try a backbone that doesn't accept 1024 wide tokens (like BigBird or LongFormer), then we can add a sliding window to train and inference. BigBird is a new SOTA transformer with arXiv paper [here][3] which can accept large token inputs as wide as 4096!

The model in this notebook uses HuggingFace's `AutoModelForTokenClassification`. If we want a custom head, we could use `AutoModel` and then build our own head. See my TensorFlow notebook [here][2] for an example.

The tokenization process uses `tokenizer(txt.split(), is_split_into_words=True)`, note that this ignores characters like `\n`. If we want our model to see new paragraphs, we need to rewrite this code and avoid `is_split_into_words=True`. See my TensorFlow notebook [here][2] for an example.

This notebook uses many code cells from Raghavendrakotala's great notebook [here][1]. Don't forget to upvote Raghavendrakotala's notebook :-)

[1]: https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533
[2]: https://www.kaggle.com/cdeotte/tensorflow-longformer-ner-cv-0-617
[3]: https://arxiv.org/abs/2007.14062

# Configuration
This notebook can either train a new model or load a previously trained model (made from previous notebook version). Furthermore, this notebook can either create new NER labels or load existing NER labels (made from previous notebook version). In this notebook version, we will load model and load NER labels.

Also this notebook can load huggingface stuff (like tokenizers) from a Kaggle dataset, or download it from internet. (If it downloads from internet, you can then put it in a Kaggle dataset, so next time you can turn internet off).

In [1]:
from tqdm import tqdm

In [2]:
import os
# DECLARE HOW MANY GPUS YOU WISH TO USE. 
# KAGGLE ONLY HAS 1, BUT OFFLINE, YOU CAN USE MORE
os.environ["CUDA_VISIBLE_DEVICES"]="0" #0,1,2,3 for four gpu

# VERSION FOR SAVING MODEL WEIGHTS
TRAIN_VER = 3
LOAD_VER = 3

# IF VARIABLE IS NONE, THEN NOTEBOOK COMPUTES TOKENS
# OTHERWISE NOTEBOOK LOADS TOKENS FROM PATH
LOAD_TOKENS_FROM = 'NER_token'

# IF VARIABLE IS NONE, THEN NOTEBOOK TRAINS A NEW MODEL
# OTHERWISE IT LOADS YOUR PREVIOUSLY TRAINED MODEL
LOAD_MODEL_FROM = 'trained_model'
# LOAD_MODEL_FROM = None

# IF FOLLOWING IS NONE, THEN NOTEBOOK 
# USES INTERNET AND DOWNLOADS HUGGINGFACE 
# CONFIG, TOKENIZER, AND MODEL
# DOWNLOADED_MODEL_PATH = '../input/py-bigbird-v26' 
DOWNLOADED_MODEL_PATH = None

if DOWNLOADED_MODEL_PATH is None:
    DOWNLOADED_MODEL_PATH = 'model'    
MODEL_NAME = 'google/bigbird-roberta-base'

In [3]:
from torch import cuda
config = {'model_name': MODEL_NAME,
         'max_length': 1024,
         # 'train_batch_size':4,
         # 'valid_batch_size':4,
         'train_batch_size':3,
         'valid_batch_size':3,
         'epochs':5,
         'learning_rates': [2.5e-5, 2.5e-5, 2.5e-6, 2.5e-6, 2.5e-7],
         'max_grad_norm':10,
         'device': 'cuda' if cuda.is_available() else 'cpu'}

# THIS WILL COMPUTE VAL SCORE DURING COMMIT BUT NOT DURING SUBMIT
COMPUTE_VAL_SCORE = True
if len( os.listdir('../data/test') ) > 5:
      COMPUTE_VAL_SCORE = False

# How To Submit PyTorch Without Internet
Many people ask me, how do I submit PyTorch models without internet? With HuggingFace Transformer, it's easy. Just download the following 3 things (1) model weights, (2) tokenizer files, (3) config file, and upload them to a Kaggle dataset. Below shows code how to get the files from HuggingFace for Google's BigBird-base. But this same code can download any transformer, like for example roberta-base.

In [4]:
from transformers import *
if DOWNLOADED_MODEL_PATH == 'model' and not os.path.exists('model'):
    os.mkdir('model')
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, add_prefix_space=True)
    tokenizer.save_pretrained('model')

    config_model = AutoConfig.from_pretrained(MODEL_NAME) 
    config_model.num_labels = 15
    config_model.save_pretrained('model')

    backbone = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, 
                                                               config=config_model)
    backbone.save_pretrained('model')

L:\Anaconda3\lib\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


# Load Data and Libraries
In addition to loading the train dataframe, we will load all the train and text files and save them in a dataframe.

In [5]:
import numpy as np, os 
import pandas as pd, gc 
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForTokenClassification
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.metrics import accuracy_score

In [6]:
train_df = pd.read_csv('../data/train.csv')
print( train_df.shape )
train_df.head()

(144293, 8)


,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [7]:
# https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533
test_names, test_texts = [], []
for f in list(os.listdir('../data/test')):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('../data/test/' + f, 'r').read())
test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
test_texts.head()

,id,text
0,0FB0700DAF44,"During a group project, have you ever asked a ..."
1,18409261F5C2,80% of Americans believe seeking multiple opin...
2,D46BCB48440A,"When people ask for advice,they sometimes talk..."
3,D72CB1C11673,Making choices in life can be very difficult. ...
4,DF920E0A7337,Have you ever asked more than one person for h...


In [8]:
# https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533
test_names, train_texts = [], []
for f in tqdm(list(os.listdir('../data/train'))):
    test_names.append(f.replace('.txt', ''))
    train_texts.append(open('../data/train/' + f, 'r').read())
train_text_df = pd.DataFrame({'id': test_names, 'text': train_texts})
train_text_df.head()

100%|██████████| 15594/15594 [00:03<00:00, 4365.04it/s]


,id,text
0,0000D23A521A,"Some people belive that the so called ""face"" o..."
1,00066EA9880D,Driverless cars are exaclty what you would exp...
2,000E6DE9E817,Dear: Principal\n\nI am arguing against the po...
3,001552828BD0,Would you be able to give your car up? Having ...
4,0016926B079C,I think that students would benefit from learn...


# Convert Train Text to NER Labels
We will now convert all text words into NER labels and save in a dataframe.

In [9]:
if not os.path.exists(f'{LOAD_TOKENS_FROM}/train_NER.csv'):
    all_entities = []
    for ii, i in enumerate(train_text_df.iterrows()):
        if ii%100==0: print(ii, ', ' ,end='')
        total = i[1]['text'].split().__len__()
        entities = ["O"] * total
        # print(total)
        for j in train_df[train_df['id'] == i[1]['id']].iterrows():
            discourse = j[1]['discourse_type']
            list_ix = [int(x) for x in j[1]['predictionstring'].split(' ')]
            entities[list_ix[0]] = f"B-{discourse}"
            for k in list_ix[1:]: 
                # entities[k] = f"I-{discourse}"
                try:
                    entities[k] = f"I-{discourse}"
                except:
                    print(len(entities))
                    print(k)
        all_entities.append(entities)
    train_text_df['entities'] = all_entities
    train_text_df.to_csv('train_NER.csv',index=False)
    
else:
    from ast import literal_eval
    train_text_df = pd.read_csv(f'{LOAD_TOKENS_FROM}/train_NER.csv')
    # pandas saves lists as string, we must convert back
    train_text_df.entities = train_text_df.entities.apply(lambda x: literal_eval(x) )
    
print( train_text_df.shape )
train_text_df.head()

(15594, 3)


,id,text,entities
0,E112CE5938F4,Segoing Cowboys is a good program that I think...,"[B-Position, I-Position, I-Position, I-Positio..."
1,13326EED1708,"Yes I agree that Venus is dangerous place, but...","[B-Position, I-Position, I-Position, I-Positio..."
2,DD3F7252F887,Facial action coding sytsem should be used bec...,"[B-Position, I-Position, I-Position, I-Positio..."
3,E676D0FB128A,"My fellow citizens, there are many reasons why...","[O, O, O, B-Position, I-Position, I-Position, ..."
4,9A03E46C0A51,"Dear Principal,\n\nI believe that changing the...","[O, O, B-Position, I-Position, I-Position, I-P..."


In [10]:
# CREATE DICTIONARIES THAT WE CAN USE DURING TRAIN AND INFER
output_labels = ['O', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

In [11]:
labels_to_ids

{'O': 0,
 'B-Lead': 1,
 'I-Lead': 2,
 'B-Position': 3,
 'I-Position': 4,
 'B-Claim': 5,
 'I-Claim': 6,
 'B-Counterclaim': 7,
 'I-Counterclaim': 8,
 'B-Rebuttal': 9,
 'I-Rebuttal': 10,
 'B-Evidence': 11,
 'I-Evidence': 12,
 'B-Concluding Statement': 13,
 'I-Concluding Statement': 14}

In [12]:
# text = train_dataset['text'][0]

In [13]:
# pre_text = '[CLS] ' + text.replace('.', '. [SEP]')
# encoding = tokenizer(pre_text.split(),
#                     is_split_into_words=True,
#                     add_special_tokens=False,
#                     return_offsets_mapping=True, 
#                     padding='max_length', 
#                     truncation=True, 
#                     max_length=1024)

In [14]:
# encoding.word_ids()

In [15]:
# encoding['input_ids']

# Define the dataset function
Below is our PyTorch dataset function. It always outputs tokens and attention. During training it also provides labels. And during inference it also provides word ids to help convert token predictions into word predictions.

Note that we use `text.split()` and `is_split_into_words=True` when we convert train text to labeled train tokens. This is how the HugglingFace tutorial does it. However, this removes characters like `\n` new paragraph. If you want your model to see new paragraphs, then we need to map words to tokens ourselves using `return_offsets_mapping=True`. See my TensorFlow notebook [here][1] for an example.

Some of the following code comes from the example at HuggingFace [here][2]. However I think the code at that link is wrong. The HuggingFace original code is [here][3]. With the flag `LABEL_ALL` we can either label just the first subword token (when one word has more than one subword token). Or we can label all the subword tokens (with the word's label). In this notebook version, we label all the tokens. There is a Kaggle discussion [here][4]

[1]: https://www.kaggle.com/cdeotte/tensorflow-longformer-ner-cv-0-617
[2]: https://huggingface.co/docs/transformers/custom_datasets#tok_ner
[3]: https://github.com/huggingface/transformers/blob/86b40073e9aee6959c8c85fcba89e47b432c4f4d/examples/pytorch/token-classification/run_ner.py#L371
[4]: https://www.kaggle.com/c/feedback-prize-2021/discussion/296713

In [16]:
LABEL_ALL_SUBTOKENS = True

class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, get_wids):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.get_wids = get_wids # for validation

    def __getitem__(self, index):
        # GET TEXT AND WORD LABELS 
        text = self.data.text[index]        
        word_labels = self.data.entities[index] if not self.get_wids else None

        # TOKENIZE TEXT
        text = '[CLS] ' + text.replace('. ', '. [SEP] ')
        encoding = self.tokenizer(text.split(),
                             is_split_into_words=True,
                             add_special_tokens=False,
                             #return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        word_ids = encoding.word_ids()  
        input_ids = encoding['input_ids']
        
        # CREATE TARGETS
        if not self.get_wids:
            previous_word_idx = None
            sen_count = 1
            label_ids = []
            sen_label_ids = []
            
            for i, word_idx in enumerate(word_ids):                            
                if word_idx is None:
                    label_ids.append(-100)
                elif input_ids[i] == 65:
                    label_ids.append(-100)
                elif input_ids[i] == 66:
                    label_ids.append( max(sen_label_ids, key=sen_label_ids.count) )
                    sen_label_ids = []
                    sen_count += 1
                elif word_idx != previous_word_idx:              
                    label_ids.append( labels_to_ids[word_labels[word_idx-sen_count]] )
                    sen_label_ids.append(label_ids[-1])
                else:
                    if LABEL_ALL_SUBTOKENS:
                        label_ids.append( labels_to_ids[word_labels[word_idx-sen_count]] )
                        sen_label_ids.append(label_ids[-1])
                    else:
                        label_ids.append(-100)
                previous_word_idx = word_idx
            encoding['labels'] = label_ids

        # CONVERT TO TORCH TENSORS
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        if self.get_wids: 
            word_ids2 = []
            for i, w in enumerate(word_ids):
                if w is None:
                    word_ids2.append(-1)
                elif input_ids[i] == 65 or input_ids[i] == 66:
                    word_ids2.append(-1)
                else:
                    word_ids2.append(w)
            # word_ids2 = [w if w is not None else -1 for w in word_ids]
            item['wids'] = torch.as_tensor(word_ids2)
        
        return item

    def __len__(self):
        return self.len

In [17]:
from scipy import stats

# Return an array that maps character index to index of word in list of split() words
def split_mapping(unsplit):
    splt = unsplit.split()
    offset_to_wordidx = np.full(len(unsplit),-1)
    txt_ptr = 0
    for split_index, full_word in enumerate(splt):
        while unsplit[txt_ptr:txt_ptr + len(full_word)] != full_word:
            txt_ptr += 1
        offset_to_wordidx[txt_ptr:txt_ptr + len(full_word)] = split_index
        txt_ptr += len(full_word)
    return offset_to_wordidx

In [18]:
LABEL_ALL_SUBTOKENS = True

class dataset_o(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, get_wids):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.get_wids = get_wids # for validation

    def __getitem__(self, index):
        # GET TEXT AND WORD LABELS 
        text = self.data.text[index]        
        word_labels = self.data.entities[index] if not self.get_wids else None

        # TOKENIZE TEXT
        encoding = self.tokenizer(text,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        word_ids = encoding.word_ids()  
        split_word_ids = np.full(len(word_ids),-1)
        offset_to_wordidx = split_mapping(text)
        offsets = encoding['offset_mapping']
        
        # CREATE TARGETS AND MAPPING OF TOKENS TO SPLIT() WORDS
        label_ids = []
        # Iterate in reverse to label whitespace tokens until a Begin token is encountered
        for token_idx, word_idx in reversed(list(enumerate(word_ids))):
            
            if word_idx is None:
                if not self.get_wids: label_ids.append(-100)
            else:
                if offsets[token_idx] != (0,0):
                    #Choose the split word that shares the most characters with the token if any
                    split_idxs = offset_to_wordidx[offsets[token_idx][0]:offsets[token_idx][1]]
                    split_index = stats.mode(split_idxs[split_idxs != -1]).mode[0] if len(np.unique(split_idxs)) > 1 else split_idxs[0]
                    
                    if split_index != -1: 
                        if not self.get_wids: label_ids.append( labels_to_ids[word_labels[split_index]] )
                        split_word_ids[token_idx] = split_index
                    else:
                        # Even if we don't find a word, continue labeling 'I' tokens until a 'B' token is found
                        if label_ids and label_ids[-1] != -100 and ids_to_labels[label_ids[-1]][0] == 'I':
                            split_word_ids[token_idx] = split_word_ids[token_idx + 1]
                            if not self.get_wids: label_ids.append(label_ids[-1])
                        else:
                            if not self.get_wids: label_ids.append(-100)
                else:
                    if not self.get_wids: label_ids.append(-100)
        
        encoding['labels'] = list(reversed(label_ids))

        # CONVERT TO TORCH TENSORS
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        if self.get_wids: 
            item['wids'] = torch.as_tensor(split_word_ids)
        
        # https://www.kaggle.com/vuxxxx/tensorflow-longformer-ner-postprocessing#Postprocessing
#         item['offsets'] = offsets
        
        return item

    def __len__(self):
        return self.len

# Create Train and Validation Dataloaders
We will use the same train and validation subsets as my TensorFlow notebook [here][1]. Then we can compare results. And/or experiment with ensembling the validation fold predictions.

[1]: https://www.kaggle.com/cdeotte/tensorflow-longformer-ner-cv-0-617

In [19]:
# CHOOSE VALIDATION INDEXES (that match my TF notebook)
IDS = train_df.id.unique()
print('There are',len(IDS),'train texts. We will split 90% 10% for validation.')

# TRAIN VALID SPLIT 90% 10%
np.random.seed(42)
train_idx = np.random.choice(np.arange(len(IDS)),int(0.9*len(IDS)),replace=False)
valid_idx = np.setdiff1d(np.arange(len(IDS)),train_idx)
np.random.seed(None)

There are 15594 train texts. We will split 90% 10% for validation.


In [20]:
# CREATE TRAIN SUBSET AND VALID SUBSET
data = train_text_df[['id','text', 'entities']]
train_dataset = data.loc[data['id'].isin(IDS[train_idx]),['text', 'entities']].reset_index(drop=True)
test_dataset = data.loc[data['id'].isin(IDS[valid_idx])].reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

tokenizer = AutoTokenizer.from_pretrained(DOWNLOADED_MODEL_PATH) 
training_set = dataset(train_dataset, tokenizer, config['max_length'], False)
testing_set = dataset(test_dataset, tokenizer, config['max_length'], True)
# testing_set = dataset_o(test_dataset, tokenizer, config['max_length'], True)

Didn't find file model\added_tokens.json. We won't load it.
loading file model\spiece.model
loading file model\tokenizer.json
loading file None
loading file model\special_tokens_map.json
loading file model\tokenizer_config.json


FULL Dataset: (15594, 3)
TRAIN Dataset: (14034, 2)
TEST Dataset: (1560, 3)


In [21]:
# TRAIN DATASET AND VALID DATASET
train_params = {'batch_size': config['train_batch_size'],
                'shuffle': True,
                # 'num_workers': 2,
                'num_workers': 0,
                'pin_memory':True
               }

test_params = {'batch_size': config['valid_batch_size'],
               'shuffle': False,
               # 'num_workers': 2,
               'num_workers': 0,
               'pin_memory':True
              }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

# TEST DATASET
test_texts_set = dataset(test_texts, tokenizer, config['max_length'], True)
test_texts_loader = DataLoader(test_texts_set, **test_params)

In [22]:
training_set[0]

{'input_ids': tensor([  65, 1102, 5247,  ...,    0,    0,    0]),
 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]),
 'labels': tensor([-100,    3,    3,  ..., -100, -100, -100])}

In [23]:
tokenizer.decode(training_set[0]['input_ids'][:360])

"[CLS] Segoing Cowboys is a good program that I think everybody should join.[SEP] In the program, you can do things that a cowboy usually does, but you can also do other fun things.[SEP] Here are some other reasons for you to join. If you like animals, this job is for you.[SEP] One of the main reasons this program started was to help countries with animals, and that is a fauna-lover's dream.[SEP] Another reason is that you can see the different sights in other coutries.[SEP] The sights are absolutely amazing! You will be able to see the beautiful sunset on the sea and there is pretty much nothing prettier than that.[SEP] But, if that hasn't changed your minds about not going, then this certainly will. Some people say that seafaring is dangerous, or that you will not make it out alive if the ship sinks.[SEP] First of all, there is absolutely no way that you will get killed on the ship, and second, there is about a fourteen million to one chance that the ship will sink, because our capta

In [24]:
tokenizer.decode(training_set[0]['input_ids'][18])

'program'

In [25]:
training_set[0]['labels'][18]

tensor(4)

In [26]:
training_set[0]['labels'][:360]

tensor([-100,    3,    3,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
           4,    4,    5,    6,    6,    6,    6,    6,    6,    6,    6,    6,
           6,    4,   11,   12,   12,   12,   12,   12,   12,   12,   12,   12,
          12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,
          12,   12,   12,   12,   12,   12,    0,    0,    0,    0,    5,    6,
           6,    6,    6,    6,    6,    6,    6,    6,    6,    6,   11,   12,
          12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,
          12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,
          12,   12,   12,   12,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   

In [27]:
text = train_dataset['text'][0]
pre_text = '[CLS] ' + text.replace('. ', '. [SEP] ')

# Train Model
The PyTorch train function is taken from Raghavendrakotala's great notebook [here][1]. I assume it uses a masked loss which avoids computing loss when target is `-100`. If not, we need to update this.

In Kaggle notebooks, we will train our model for 5 epochs `batch_size=4` with Adam optimizer and learning rates `LR = [2.5e-5, 2.5e-5, 2.5e-6, 2.5e-6, 2.5e-7]`. The loaded model was trained offline with `batch_size=8` and `LR = [5e-5, 5e-5, 5e-6, 5e-6, 5e-7]`. (Note the learning rate changes `e-5`, `e-6`, and `e-7`). Using `batch_size=4` will probably achieve a better validation score than `batch_size=8`, but I haven't tried yet.

[1]: https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533

In [28]:
# https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    #tr_preds, tr_labels = [], []
    
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(tqdm(training_loader, total=len(training_loader))):
        
        ids = batch['input_ids'].to(config['device'], dtype = torch.long)
        mask = batch['attention_mask'].to(config['device'], dtype = torch.long)
        labels = batch['labels'].to(config['device'], dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels,
                               return_dict=False)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 1000==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss after {idx:04d} training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        #tr_labels.extend(labels)
        #tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=config['max_grad_norm']
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [29]:
# CREATE MODEL
config_model = AutoConfig.from_pretrained(DOWNLOADED_MODEL_PATH+'/config.json') 
model = AutoModelForTokenClassification.from_pretrained(
                   DOWNLOADED_MODEL_PATH+'/pytorch_model.bin',config=config_model)
model.to(config['device'])
optimizer = torch.optim.Adam(params=model.parameters(), lr=config['learning_rates'][0])

loading configuration file model/config.json
Model config BigBirdConfig {
  "_name_or_path": "model/config.json",
  "architectures": [
    "BigBirdForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "attention_type": "block_sparse",
  "block_size": 64,
  "bos_token_id": 1,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
  

In [30]:
from torch.nn import CrossEntropyLoss

def my_forward(
    self=model,
    input_ids=None,
    attention_mask=None,
    token_type_ids=None,
    position_ids=None,
    head_mask=None,
    inputs_embeds=None,
    labels=None,
    output_attentions=None,
    output_hidden_states=None,
    return_dict=None,
):
    r"""
    labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
    Labels for computing the token classification loss. Indices should be in `[0, ..., config.num_labels - 1]`.
    """
    return_dict = return_dict if return_dict is not None else self.config.use_return_dict

    outputs = self.bert(
        input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids,
        position_ids=position_ids,
        head_mask=head_mask,
        inputs_embeds=inputs_embeds,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict,
    )

    sequence_output_s = outputs[0] # [bs, 1024, 768]
#     print('233')
#     print(outputs[0][:,0,:].shape)
#     print(outputs[0][:,0,:])
#     print(outputs[0].shape)
#     print(outputs[0])
    # print(input_ids.shape)
    
    # index = torch.nonzero(input_ids==66)
    # sequence_output_s = outputs[0][index[:,0], index[:,1], :]
    # print(index.shape)
    # print(sequence_output_s.shape)
    
#     shape = sequence_output[0].shape
#     index = torch.nonzero(input_ids[0]==66).flatten()
#     sequence_output_s = input_ids[0][index].view(-1, 1,shape[0],shape[1])

#     for idx,batch in enumerate(input_ids[1:]):
#         index = torch.nonzero(batch==66).flatten()
#         torch.cat([sequence_output_s,sequence_output[idx+1][index].view(1, len(index), dim)],dim=0)
    
    sequence_output_s = self.dropout(sequence_output_s)
    logits = self.classifier(sequence_output_s)

    loss = None
    if labels is not None:
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
    
    # print(self.num_labels) # 15
    # print(logits.view(-1, self.num_labels).shape) # [bs x 1024, 15]
    # print(labels.shape) # [bs, 1024]
    # print(loss)

    if not return_dict:
        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

    return TokenClassifierOutput(
        loss=loss,
        logits=logits,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions,
    )

In [31]:
# model.forward = my_forward

In [32]:
## ignore __floordiv__ is deprecated
import warnings
warnings.filterwarnings("ignore")

# LOOP TO TRAIN MODEL (or load model)
if not LOAD_MODEL_FROM:
    for epoch in range(config['epochs']):
        
        print(f"### Training epoch: {epoch + 1}")
        for g in optimizer.param_groups: 
            g['lr'] = config['learning_rates'][epoch]
        lr = optimizer.param_groups[0]['lr']
        print(f'### LR = {lr}\n')
        
        train(epoch)
        torch.cuda.empty_cache()
        gc.collect()
        
    torch.save(model.state_dict(), f'bigbird_v{TRAIN_VER}.pt')
else:
    model.load_state_dict(torch.load(f'{LOAD_MODEL_FROM}/bigbird_v{LOAD_VER}.pt'))
    print('Model loaded.')

Model loaded.


# Inference and Validation Code
We will infer in batches using our data loader which is faster than inferring one text at a time with a for-loop. The metric code is taken from Rob Mulla's great notebook [here][2]. Our model achieves validation F1 score 0.615! 

During inference our model will make predictions for each subword token. Some single words consist of multiple subword tokens. In the code below, we use a word's first subword token prediction as the label for the entire word. We can try other approaches, like averaging all subword predictions or taking `B` labels before `I` labels etc.

[1]: https://www.kaggle.com/raghavendrakotala/fine-tunned-on-roberta-base-as-ner-problem-0-533
[2]: https://www.kaggle.com/robikscube/student-writing-competition-twitch

In [33]:
def inference(batch):
                
    # MOVE BATCH TO GPU AND INFER
    ids = batch["input_ids"].to(config['device'])
    mask = batch["attention_mask"].to(config['device'])
    outputs = model(ids, attention_mask=mask, return_dict=False)
    all_preds = torch.argmax(outputs[0], axis=-1).cpu().numpy() 

    # INTERATE THROUGH EACH TEXT AND GET PRED
    predictions = []
    for k,text_preds in enumerate(all_preds):
        token_preds = [ids_to_labels[i] for i in text_preds]

        prediction = []
        word_ids = batch['wids'][k].numpy()  
        previous_word_idx = -1
        for idx,word_idx in enumerate(word_ids):                            
            if word_idx == -1:
                pass
            elif word_idx != previous_word_idx:              
                prediction.append(token_preds[idx])
                previous_word_idx = word_idx
        predictions.append(prediction)
    
    return predictions

In [34]:
# https://www.kaggle.com/zzy990106/pytorch-ner-infer
# code has been modified from original
def get_predictions(df=test_dataset, loader=testing_loader):
    
    # put model in training mode
    model.eval()
    
    # GET WORD LABEL PREDICTIONS
    y_pred2 = []
    for batch in tqdm(loader, total=len(loader)):
        labels = inference(batch)
        y_pred2.extend(labels)

    final_preds2 = []
    for i in tqdm(range(len(df))):

        idx = df.id.values[i]
        #pred = [x.replace('B-','').replace('I-','') for x in y_pred2[i]]
        pred = y_pred2[i] # Leave "B" and "I"
        preds = []
        j = 0
        while j < len(pred):
            cls = pred[j]
            if cls == 'O': j += 1
            else: cls = cls.replace('B','I') # spans start with B
            end = j + 1
            while end < len(pred) and pred[end] == cls:
                end += 1
            
            if cls != 'O' and cls != '' and end - j > 7:
                final_preds2.append((idx, cls.replace('I-',''),
                                     ' '.join(map(str, list(range(j, end))))))
        
            j = end
        
    oof = pd.DataFrame(final_preds2)
    oof.columns = ['id','class','predictionstring']

    return oof

In [35]:
# from Rob Mulla @robikscube
# https://www.kaggle.com/robikscube/student-writing-competition-twitch
def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]


def score_feedback_comp(pred_df, gt_df):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df[['id','discourse_type','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df = pred_df[['id','class','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')

    joined['overlaps'] = joined.apply(calc_overlap, axis=1)

    # 2. If the overlap between the ground truth and prediction is >= 0.5, 
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])


    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)
    tp_pred_ids = joined.query('potential_TP') \
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','predictionstring_gt']).first()['pred_id'].values

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids]

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    #calc microf1
    my_f1_score = TP / (TP + 0.5*(FP+FN))
    return my_f1_score

In [36]:
if COMPUTE_VAL_SCORE: # note this doesn't run during submit
    # VALID TARGETS
    valid = train_df.loc[train_df['id'].isin(IDS[valid_idx])]

    # OOF PREDICTIONS
    oof = get_predictions(test_dataset, testing_loader)

    # COMPUTE F1 SCORE
    f1s = []
    CLASSES = oof['class'].unique()
    print()
    for c in tqdm(CLASSES):
        pred_df = oof.loc[oof['class']==c].copy()
        gt_df = valid.loc[valid['discourse_type']==c].copy()
        f1 = score_feedback_comp(pred_df, gt_df)
        print(c, f1)
        f1s.append(f1)
    print()
    print('Overall', np.mean(f1s))
    print()

100%|██████████| 1560/1560 [00:00<00:00, 6830.45it/s]


 14%|█▍        | 1/7 [00:00<00:00,  9.03it/s]

Position 0.6595460614152203


 29%|██▊       | 2/7 [00:00<00:02,  2.26it/s]

Claim 0.5308546914530855


 57%|█████▋    | 4/7 [00:01<00:01,  2.31it/s]

Evidence 0.7119448037947391
Concluding Statement 0.7630252100840336
Counterclaim 0.5032139577594124
Rebuttal 0.402088772845953


100%|██████████| 7/7 [00:01<00:00,  3.59it/s]

Lead 0.7831196581196581

Overall 0.6219704507817289



In [37]:
oof['class'].unique()

array(['Position', 'Claim', 'Evidence', 'Concluding Statement',
       'Counterclaim', 'Rebuttal', 'Lead'], dtype=object)

In [38]:
oof.loc[oof['id']=='DD3F7252F887']

,id,class,predictionstring
0,DD3F7252F887,Position,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
1,DD3F7252F887,Claim,18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 3...
2,DD3F7252F887,Evidence,52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 6...
3,DD3F7252F887,Claim,145 146 147 148 149 150 151 152 153 154 155 15...
4,DD3F7252F887,Evidence,161 162 163 164 165 166 167 168 169 170 171 17...
5,DD3F7252F887,Evidence,262 263 264 265 266 267 268 269 270 271 272 27...
6,DD3F7252F887,Concluding Statement,336 337 338 339 340 341 342 343 344 345 346 34...


In [39]:
oof['id'][9]

'F6D56988EB7D'

In [40]:
oof['predictionstring'][9]

'119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134'

In [41]:
pred_df = oof.loc[oof['class']==c].copy()

In [42]:
valid.loc[valid['id']=='DD3F7252F887']

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
53662,DD3F7252F887,1.619534e+12,0.0,97.0,Facial action coding sytsem should be used bec...,Position,Position 1,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
53663,DD3F7252F887,1.619534e+12,97.0,211.0,First off i think if this is being used on stu...,Claim,Claim 1,18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 3...
53664,DD3F7252F887,1.619534e+12,212.0,408.0,and they are to scared to ask for help and in ...,Evidence,Evidence 1,41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 5...
53665,DD3F7252F887,1.619534e+12,408.0,500.0,Also if a student is bored this will make the ...,Claim,Claim 2,78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 9...
53666,DD3F7252F887,1.619535e+12,501.0,579.0,This is a good way to help people to what if t...,Claim,Claim 3,98 99 100 101 102 103 104 105 106 107 108 109 ...
53667,DD3F7252F887,1.619535e+12,579.0,740.0,and they need help or want to talk to somebody...,Evidence,Evidence 2,114 115 116 117 118 119 120 121 122 123 124 12...
53668,DD3F7252F887,1.619535e+12,741.0,900.0,Plus it is cool because you can find out alot...,Claim,Claim 4,145 146 147 148 149 150 151 152 153 154 155 15...
53669,DD3F7252F887,1.619535e+12,901.0,1425.0,This is really cool beacsue you can find out a...,Evidence,Evidence 3,178 179 180 181 182 183 184 185 186 187 188 18...
53670,DD3F7252F887,1.619535e+12,1426.0,1822.0,We have expressions every day but sometimes ca...,Concluding Statement,Concluding Statement 1,282 283 284 285 286 287 288 289 290 291 292 29...


In [43]:
a = valid.loc[valid['id']=='DD3F7252F887']

In [44]:
a['predictionstring'][53670]

'282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 356 357'

In [45]:
pred_df['id'][9]

KeyError: 9

# Infer Test Data and Write Submission CSV
We will now infer the test data and write submission CSV

In [ ]:
sub = get_predictions(test_texts, test_texts_loader)
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)

# Test

In [28]:
import transformers
print(transformers.__version__)

4.16.2


In [30]:
import tokenizers
print(tokenizers.__version__)

0.11.0


In [26]:
print(torch. __version__)

1.10.0


In [24]:
a = np.array([[[1,2]]])

In [25]:
a.shape

(1, 1, 2)

In [26]:
a[0][0][0]

1

In [28]:
a.nonzero()

(array([0, 0], dtype=int64),
 array([0, 0], dtype=int64),
 array([0, 1], dtype=int64))